# The Agave API

In this notebook we explore the Agave API and use it to store and retrieve files. We will use the requests library to make all requests. For full documentation on the Agave API, see: http://developer.agaveapi.co/

To get started, we need to generate a set of Agave developer client keys (OAuth credentials). Generating OAuth clients uses HTTP Basic Authentication (https://tools.ietf.org/html/rfc2617) with your TACC username and password.

In [4]:
# import the requests library
import requests

# import getpass to prompt for a password
from getpass import getpass

In [5]:
# the base URL for interacting with the Agave API
base_url = 'https://api.tacc.utexas.edu'

In [6]:
# Set up your TACC credentials. Modify the username appropriately
username = 'jstubbs'
password = getpass(prompt='Hello {}. Please enter your TACC password: '.format(username))

Hello jstubbs. Please enter your TACC password: ········


## OAuth Client And Access Token Generation

With the username and password in place, we are ready to interact with Agave's OAuth server to generate an OAuth client and then an access token. Agave has a full OAuth provider server and supports 4 major grant types: password, authorization_code, refresh_token and implicit. For more details on OAuth see the spec (https://tools.ietf.org/html/rfc6749).

### OAuth Client Generation

We can use Agave's clients service to generate and manage our OAuth clients. First let's make a GET request to the clients service to see what clients we have. We will use the provided HTTPBasicAuth class in requests, which comes with a convenient shortcut:

In [16]:
rsp = requests.get(url='{}/clients/v2'.format(base_url), auth=(username, password))
rsp.status_code

200

In [17]:
# the clients service, like all Agave services, returns us JSON:
rsp.json()

{'message': 'Clients retrieved successfully.',
 'result': [{'_links': {'self': {'href': 'https://api.tacc.utexas.edu/clients/v2/DefaultApplication'},
    'subscriber': {'href': 'https://api.tacc.utexas.edu/profiles/v2/jstubbs'},
    'subscriptions': {'href': 'https://api.tacc.utexas.edu/clients/v2/DefaultApplication/subscriptions/'}},
   'callbackUrl': None,
   'consumerKey': 'VzN6Cxj9GirfMsINqUVQxbhrQSQa',
   'description': None,
   'name': 'DefaultApplication',
   'tier': 'Unlimited'},
  {'_links': {'self': {'href': 'https://api.tacc.utexas.edu/clients/v2/postman'},
    'subscriber': {'href': 'https://api.tacc.utexas.edu/profiles/v2/jstubbs'},
    'subscriptions': {'href': 'https://api.tacc.utexas.edu/clients/v2/postman/subscriptions/'}},
   'callbackUrl': 'https://www.getpostman.com/oauth2/callback',
   'consumerKey': 'fYEbfnaxo4LbqShSebng4f5LD18a',
   'description': '',
   'name': 'postman',
   'tier': 'Unlimited'},
  {'_links': {'self': {'href': 'https://api.tacc.utexas.edu/client

To create a simple OAuth client that has access to all basic Agave APIs, we need to make a POST request to the clients service. The only required field we need to pass in is `clientName` to give a name to our client. Each client we create must have a unique name.

In [18]:
# Pick a name for your client; this name will have to be different every time you run this cell. Otherwise, you
# will try to recreate a client with the same name and you will get an error.
client_name = 'cic_institute'

# make a POST request to the client's service, passing only that field. 
# Note that the parameter name uses camel case
data = {'clientName': client_name}
rsp = requests.post(url='{}/clients/v2'.format(base_url), data=data, auth=(username, password))
rsp.status_code

201

Note that Agave returned a 201 status code to indicate that resource was successfully created. 

Let's explore the response.

In [19]:
rsp.json()

{'message': 'Client created successfully.',
 'result': {'_links': {'self': {'href': 'https://api.tacc.utexas.edu/clients/v2/cic_institute'},
   'subscriber': {'href': 'https://api.tacc.utexas.edu/profiles/v2/jstubbs'},
   'subscriptions': {'href': 'https://api.tacc.utexas.edu/clients/v2/cic_institute/subscriptions/'}},
  'callbackUrl': '',
  'consumerKey': 'f_9vaEvm3oxCnNe9Z4VoxRXZGwca',
  'consumerSecret': 'MTXYtMiwOmxNI2mRLrwOwG_9Vpwa',
  'description': '',
  'name': 'cic_institute',
  'tier': 'Unlimited'},
 'status': 'success',
 'version': '2.0.0-SNAPSHOT-rc3fad'}

Two important string fields are returned, the `consumerKey` and the `consumerSecret`. We will need these fields to interact with the Agave OAuth token service and generate an access token. Let's take note of those variables.

In [21]:
key = rsp.json()['result']['consumerKey']

In [22]:
secret = rsp.json()['result']['consumerSecret']

### Generating Access and Refresh Tokens

We're now ready to generate an OAuth token. This token will represent both the user and client application. In this case, they are owned by the same individual, but in general they will not be.

To generate an OAuth token, we make a POST request to Agave's token service. We have to pass in several fields to make the request:

In [23]:
# POST payload for generating a token using the password grant:
# scope will always be PRODUCTION.
data = {'username': username,
       'password': password,
       'grant_type': 'password',
       'scope': 'PRODUCTION'}
# note that authentication is technically HTTPBasicAuth with the OAuth client key and secret
rsp = requests.post('{}/token'.format(base_url), data=data, auth=(key, secret))
rsp.status_code

200

In [24]:
# check the response message:
rsp.json()

{'access_token': '6f8f0b94c413c8a7b5ba428bd50d4',
 'expires_in': 14400,
 'refresh_token': 'dafb6ab783b8edd2cfceda40a7f134',
 'scope': 'default',
 'token_type': 'bearer'}

We see that Agave generated an access token and a refresh token. The access token is good for 4 hours (14400 seconds). But we can use the refresh token to get a new access token at any point. To do so, we will use the refresh_token grant type and a modified payload. For now, let's just grab the tokens from the response.

In [26]:
access_token = rsp.json()['access_token']
refresh_token = rsp.json()['refresh_token']

Once we have an access token we are ready to interact with the rest of the Agave services. All requests to Agave using this access token will be done on behalf of the user who's credentials were used to retrieve the token (as well as the OAuth client that was used).

In order to make a request to Agave using the access token, we need to pass the token into the Authorization header of the request. The value of the header must be formated like so: `"Bearer <access_token>"`

As a simple check, we'll use the Agave Profiles service to pull the "profile" associated with this token. The Agave Profiles service maintains some details about registered users.

In [27]:
# build the Authorization header in a headers dictionary
headers = {'Authorization': 'Bearer {}'.format(access_token)}

# make a request to the profiles service; the "me" endpoint is a special reserved word in Agave to indicate
# we want information about the associated token.
rsp = requests.get(url='{}/profiles/v2/me'.format(base_url), headers=headers)
rsp.status_code

200

In [28]:
rsp.json()

{'message': 'User details retrieved successfully.',
 'result': {'create_time': '20140515180254Z',
  'email': 'jstubbs@tacc.utexas.edu',
  'first_name': 'Joe',
  'full_name': 'jstubbs',
  'last_name': 'Stubbs',
  'mobile_phone': '',
  'phone': '',
  'status': '',
  'username': 'jstubbs'},
 'status': 'success',
 'version': '2.0.0-SNAPSHOT-rc3fad'}

Indeed, the profile belongs to me. We are now ready to interact with Agave's cloud storage.